### Import Library and Data

In [ ]:
import pandas as pd 
import pathlib
import os
from docx import Document
from docx.text.parfmt import ParagraphFormat 
from docx.shared import Inches, RGBColor

NUMBER_OF_PEOPLE = 5

In [ ]:
#import data
path = r"C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Data Set\Filtered and SQL\Filtered MDF.csv"
df = pd.read_csv(path)

df.columns
# df = df.sample(frac=1).reset_index(drop=True)
# df["Number_of_Paragraphs"]

In [ ]:
df.drop_duplicates(subset='url').shape[0]

### Functions

In [ ]:
### Source: https://github.com/python-openxml/python-docx/issues/74#issuecomment-261169410

import docx
def add_hyperlink(paragraph, url, text, color, underline):
    """
    A function that places a hyperlink within a paragraph object.

    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Add color if it is given
    if not color is None:
      c = docx.oxml.shared.OxmlElement('w:color')
      c.set(docx.oxml.shared.qn('w:val'), color)
      rPr.append(c)

    # Remove underlining if it is requested
    if not underline:
      u = docx.oxml.shared.OxmlElement('w:u')
      u.set(docx.oxml.shared.qn('w:val'), 'none')
      rPr.append(u)

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

# # extract links from hyperlink text in  
# def extract_links(file_path):
#   # Open the .docx file
#   doc = docx.Document(file_path)
  
#   # Initialize an empty list to store the links
#   links = []
  
#   # Iterate through all the paragraphs in the document
#   for para in doc.paragraphs:
#       # Iterate through all the runs in the paragraph
#       for run in para.runs:
#           # Check if the run has a hyperlink
#           if run.hyperlink:
#               # If it does, add the hyperlink to the list of links
#               links.append(run.hyperlink.address)
  
#   # Return the list of links
#   return links

# def extract_title(file_path):
#     # Open the .docx file
#     doc = docx.Document(file_path)
    
#     # Iterate through all the paragraphs in the document
#     for para in doc.paragraphs:
#         # Get the text of the paragraph
#         text = para.text
        
#         # Split the text into lines
#         lines = text.split("\n")
        
#         # Iterate through all the lines
#         for line in lines:
#             # Check if the line starts with "Title:"
#             if line.startswith("Title:"):
#                 # If it does, extract the text after "Title:" and return it
#                 title = line.split("Title:")[1].strip()
#                 return title
    
#     # If no "Title:" is found, return None
#     return None

def extract_title(text):
    # Split the text into lines
    lines = text.split("\n")
    
    # Iterate through all the lines
    for line in lines:
        # Check if the line starts with "Title:"
        if line.startswith("Title:"):
            # If it does, extract the text after "Title:" and return it
            title = line.split("Title:")[1].strip()
            return title
    
    # If no "Title:" is found, return None
    return None


In [ ]:

# Example usage
text = "Post Url: Link to Post, Subreddit: r/college, Title: Seeking input on my list of potential transfer colleges"
# title = extract_title(text)

lines = text.split("\n")
lines



### Creating Assignment Docs

In [ ]:
"""
Note: Remember to rerun this before every rerun of the cell below. This because the document object needs to be renewed for every time this is done. 
I will definitely make this a function once I finalize everything else. 
"""

#create document files for each subreddit in 
docs = []
    
[docs.append(Document()) for i in range(NUMBER_OF_PEOPLE)] 

#create dataframe for each person on the team
dfs = []

#randomize indexes to adjust for the length of =

index_num = [x for x in range(0, df.shape[0]+1,df.shape[0]//NUMBER_OF_PEOPLE)]

for i in range(0, len(index_num)-1, 1): #could be improved
    dfs.append(df.iloc[index_num[i]:index_num[i+1]])


In [ ]:
count = 0 
for i in range(len(dfs)): 

    for index in range(len(dfs[i])): 
        # first line of each narrative which contains the link to the submission and the subreddit
        firstLine = docs[count].add_paragraph("Post Url: ")
        add_hyperlink(firstLine, dfs[i].iloc[index]['url'], 'Link to Post', '00a7ff', False)
        firstLine.add_run(', Subreddit: r/'+ dfs[i].iloc[index]['subreddit'])

        #paragraph and format
        para = docs[count].add_paragraph(dfs[i].iloc[index]['selftext'])
        para.paragraph_format.first_line_indent = Inches(0.5)
        para.paragraph_format.right_indent = Inches(0.5)
        para.keep_together = True
        docs[count].add_page_break() 

    docs[count].save("Person_" + str(count)+ ".docx")
    count += 1

### Create Validated Dataset Doc

* Estephanos: 10 
* Labiba: 4 
* Anubav: 15(one was a duplicate)

In [ ]:
validated_Dataframe = pd.DataFrame()

#### Get data from text

In [ ]:
#for estephanos
links = []

with open("text files\Just links_estephanos.txt", 'r') as file: 
    # Iterate through all the lines
    for line in file:
        links.append(str(line[:-2]) + "/") # remove '\n' while appending

for link in links:
    validated_Dataframe = pd.concat([validated_Dataframe, df.loc[(df['url'] == link)]])

validated_Dataframe = pd.concat([validated_Dataframe, df.loc[(df['url'] == "https://www.reddit.com/r/college/comments/ir9444/considering_dropping_out_advice/")]])

validated_Dataframe = validated_Dataframe.drop_duplicates(subset=['title'])
validated_Dataframe.shape[0]

In [ ]:
#for anubav
links_1 = []
with open("text files\\Just links_anubav.txt", 'r') as file: 
    # Iterate through all the lines
    for line in file: 
        links_1.append(str(line[:-1])) # remove '\n' while appending

links_1[-1] += "/"

for link in links_1:
    validated_Dataframe = pd.concat([validated_Dataframe, df.loc[(df['url'] == link)]])

validated_Dataframe = validated_Dataframe.drop_duplicates(subset=['title'])
validated_Dataframe.shape[0]

In [ ]:
# for labiba
links_2 = []
with open("text files\\Just links_Labiba.txt", 'r') as file: 
    # Iterate through all the lines
    for line in file: 
        links_2.append(str(line[:-1])) # remove '\n' while appending

links_2[-1] += "/"
for link in links_2:
    validated_Dataframe = pd.concat([validated_Dataframe, df.loc[(df['url'] == link)]])

validated_Dataframe = validated_Dataframe.drop_duplicates(subset=['title'])
validated_Dataframe.shape[0]

In [ ]:
# for ilyas

#### Add to docx file

In [ ]:

from docx import Document
from docx.text.parfmt import ParagraphFormat 
from docx.shared import Inches, RGBColor

validated_Dataframe = validated_Dataframe.reset_index(drop=True) # reset index of dataframe 

validation_document = Document()
for index in range(len(validated_Dataframe)): 
    #heading for each story
    firstLine = validation_document.add_paragraph("Post Url: ")
    add_hyperlink(firstLine, validated_Dataframe.iloc[index]['url'], 'Link to Post', '00a7ff', False)
    firstLine.add_run(', Subreddit: r/'+ validated_Dataframe.iloc[index]['subreddit'])
    firstLine.add_run(', Title: '+ validated_Dataframe.iloc[index]['title'])

    #paragraph and format
    para = validation_document.add_paragraph(validated_Dataframe.iloc[index]['selftext'])
    para.keep_together = True

    validation_document.add_page_break() 
# path = "C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\New Narratives"
validation_document.save("Validation Document_lack.docx")

### Analysis

In [ ]:
print(validated_Dataframe.subreddit.value_counts())